# BRKSDN-1119: The FORCE AWAKENS

This notebook goes through a set of device API examples with IOS XE and ASA virtual platforms. The goal is to demonstrate different device APIs to configure and manage network devices. This is all run inside **jupyter notebooks** (aka iPython).  A jupyter notebook lets us mix text and code and provides a simple way to experiment with Python code.

### Housekeeping: Pretty Printing 

Much of the output we'll be examining will come back in XML or JSON format.  The following functions allow us to print those objects in eyeball-friendly way.

In [77]:
from lxml import etree

def pretty_print(nc_retval):
        print(etree.tostring(etree.fromstring(nc_retval.data_xml), pretty_print=True))
        
import json

def pretty_print_j(response):
    print json.dumps(json.loads(response),sort_keys=True,indent=4, separators=(',', ': '))

## NETCONF 
### The Hard Way
#### Transport Demo

After configuring NETCONF on your router, you connect using SSH as follows:

scadora$ **ssh cisco@172.16.112.101 -p 2022 -s netconf**  
 
The router will send back scads of XML beginning with this:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<hello xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">  
<capabilities>  
<capability>urn:ietf:params:netconf:base:1.0</capability>  
<capability>urn:ietf:params:netconf:base:1.1</capability> 
<snipped for brevity/>
```

You complete the handshake by pasting the following into the SSH session:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<nc:hello xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
    <nc:capabilities>
        <nc:capability>urn:ietf:params:netconf:capability:writable-running:1.0</nc:capability>
        <nc:capability>urn:ietf:params:netconf:capability:rollback-on-error:1.0</nc:capability>
        <nc:capability>urn:ietf:params:netconf:capability:validate:1.0</nc:capability>
        <nc:capability>urn:ietf:params:netconf:capability:confirmed-commit:1.0</nc:capability>
        <nc:capability>urn:ietf:params:netconf:base:1.0</nc:capability>
        <nc:capability>urn:ietf:params:netconf:base:1.1</nc:capability>
    </nc:capabilities>
</nc:hello>]]>]]>
```

So there we have it!  A NETCONF session has been established to the router. Super-exciting!  For more details on the fascinating process of the capabilities exchange, see Section 8 of [RFC 6241](https://tools.ietf.org/html/rfc6241)

#### Get-Config Operations Demo 

Continuing from where we left off above, you can retrieve the interface configuration by doing a get-config operation on the running-config with a filter for interfaces.  You accomplish this by pasting the following into the SSH session:

```xml
#348
<?xml version="1.0" encoding="UTF-8"?>
<nc:rpc xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:a7450b46-efa0-49d0-9f5d-6b6cc54f7cd3">
    <nc:get-config>
        <nc:source>
              <nc:running/>
        </nc:source>
        <nc:filter type="subtree">
            <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
            </interfaces>
        </nc:filter>
    </nc:get-config>
</nc:rpc>
##
```

Why the #348 and ##?  Simply put, 348 is the number of octets before the ##.  It is an example of "chunk framing" which allows the send to tell the receiver how many octets to expect.  This is a feature in [NETCONF 1.1 over SSH](https://tools.ietf.org/html/rfc6242).  If you are doing NETCONF by pasting XML into an SSH session, you must manually calculate this for every message you send...which is a another good reason not to paste XML into an SSH session! 

### NETCONF With NCClient Python Library
#### Session Establishment

With the ncclient library, we can establish a NETCONF session to that same device with a simple "connect" method.  ncclient handles the ssh session for us, delivers the Hello message in the right format and puts the response into objects that are easy to manipulate.

Note: this is terrible code (no error handling, no variables, not pythonic, not reusable, etc).  It is only intended to illustrate basic functionality in as simple a ways as possible.

In [78]:
from ncclient import manager
    
xe = manager.connect(host='172.16.112.101', port=2022, username='cisco', password='cisco',
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=True)

#### Capabilities

Let's look at the router's capabilities.  This the same list of things we saw in the router's <hello> message when we did the ssh.  ncclient has conveniently put it into an iterable object for us.

In [79]:
for caps in xe.server_capabilities:
    print caps

urn:ietf:params:xml:ns:yang:smiv2:CISCO-CBP-TC-MIB?module=CISCO-CBP-TC-MIB&revision=2008-06-24
urn:ietf:params:xml:ns:yang:smiv2:RSVP-MIB?module=RSVP-MIB&revision=1998-08-25
urn:ietf:params:xml:ns:yang:smiv2:ATM-FORUM-TC-MIB?module=ATM-FORUM-TC-MIB
urn:ietf:params:xml:ns:yang:ietf-packet-fields?module=ietf-packet-fields&revision=2015-07-14
urn:ietf:params:xml:ns:yang:smiv2:CISCO-ENTITY-EXT-MIB?module=CISCO-ENTITY-EXT-MIB&revision=2008-11-24
urn:cisco:params:xml:ns:yang:cisco-qos-common?module=cisco-qos-common&revision=2015-05-09
urn:ietf:params:xml:ns:yang:smiv2:CISCO-NETSYNC-MIB?module=CISCO-NETSYNC-MIB&revision=2010-10-15
http://tail-f.com/ns/mibs/SNMPv2-MIB/200210160000Z?module=SNMPv2-MIB&revision=2002-10-16
urn:cisco:params:xml:ns:yang:cisco-environment?module=cisco-environment&revision=2015-04-09
http://tail-f.com/ns/mibs/SNMP-NOTIFICATION-MIB/200210140000Z?module=SNMP-NOTIFICATION-MIB&revision=2002-10-14
urn:ietf:params:xml:ns:yang:smiv2:CISCO-CBP-TARGET-TC-MIB?module=CISCO-CBP-T

Ok, that's a bit messy, so let's tidy it up a bit and only print the base netconf capabilities:

In [80]:
nc_caps = [c for c in xe.server_capabilities if c.startswith('urn:ietf:params:netconf')]
for c in nc_caps:
    print(c)

urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=report-all
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:xpath:1.0


#### Operations
##### ncclient get-config

It's pretty simple to get the config with ncclient.  Since I only want the interface config, I define a filter.  Otherwise, ncclient handles all the XML, the chunk-framing, the oeprations, etc. 

In [81]:
filter = '''<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">'''

c = xe.get_config(source='running', filter=('subtree', filter))

pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>GigabitEthernet1</name>
      <description>Configured via Shelly</description>
      <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
      <enabled>true</enabled>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>172.16.112.100</ip>
          <netmask>255.255.255.0</netmask>
        </address>
      </ipv4>
      <ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
    </interface>
    <interface>
      <name>GigabitEthernet2</name>
      <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
      <enabled>false</enabled>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
      <ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
    </interface>
    <interface>


##### edit-config

In [82]:
edit_data = '''
<config>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>GigabitEthernet1</name>
      <description>Configured using ncclient</description>
    </interface>
   </interfaces>
</config>
'''

filter = '''
<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
  <interface>
    <name>GigabitEthernet1</name>
    <description/>
  </interface>
</interfaces>
'''

xe.edit_config(edit_data, target='running', format='xml')
c = xe.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>GigabitEthernet1</name>
      <description>Configured using ncclient</description>
    </interface>
  </interfaces>
</data>



##### get interface statistics

In [83]:
filter = '''
<interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
  <interface>
    <name>GigabitEthernet1</name>
  </interface>
</interfaces>
'''

c = xe.get(filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>GigabitEthernet1</name>
      <oper-status>up</oper-status>
      <phys-address>00:0c:29:5a:cb:71</phys-address>
      <speed>1000000000</speed>
      <statistics>
        <in-octets>6699195</in-octets>
        <in-broadcast-pkts>0</in-broadcast-pkts>
        <in-multicast-pkts>0</in-multicast-pkts>
        <in-discards>0</in-discards>
        <in-errors>0</in-errors>
        <out-octets>2857671</out-octets>
        <out-discards>0</out-discards>
        <out-errors>0</out-errors>
        <in-pkts xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces-ext">26063</in-pkts>
        <out-pkts xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces-ext">4481</out-pkts>
      </statistics>
    </interface>
  </interfaces-state>
</data>



## RESTful APIs 
### Rest in Space
The ISS API (works in a browser):
    https://api.wheretheiss.at/v1/satellites/25544
      
The Star Wars API with curl and pretty print:  
    ```scadora$ curl -s "http://swapi.co/api/people/5/" | python -m json.tool```

Simple enough to write a script around these things:

In [84]:
import json
import requests
import time

sat_data=requests.get('https://api.wheretheiss.at/v1/satellites/25544').text
sat_dict=json.loads(sat_data)

time.sleep(2)

sat_data2=requests.get('https://api.wheretheiss.at/v1/satellites/25544').text
sat_dict2=json.loads(sat_data2)

delta_alt = sat_dict2['altitude'] - sat_dict['altitude']

person = json.loads(requests.get('http://swapi.co/api/people/5/').text)['name']

if (delta_alt > 0):
   print "The ISS is ascending! Inform " + person + "!"
elif ( delta_alt == 0 ):
   print "The ISS is stable! Don't inform " + person + "!"
else:
   print "The ISS is descending! Inform " + person + "!"

The ISS is ascending! Inform Leia Organa!


### REST API -- Get Interface Config from ASA
Start with the ASA REST API guide or, better, the onbox API browser: https://172.16.112.110/doc/  You can test APIs from here and generate code...but for a more generic tool, try Postman.

The following code was generated from Postman (mostly).  It will get interface config for Management0/0.

In [85]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

url = "https://172.16.112.110/api/interfaces/physical/Management0_API_SLASH_0"

headers = {
    'authorization': "Basic Y2lzY286Y2lzY28=",
    'content-type': "application/json"
    }

response1 = requests.request("GET", url, headers=headers, verify=False)

pretty_print_j(response1.text)

{
    "activeMacAddress": "",
    "channelGroupID": "",
    "channelGroupMode": "active",
    "duplex": "auto",
    "flowcontrolHigh": -1,
    "flowcontrolLow": -1,
    "flowcontrolOn": false,
    "flowcontrolPeriod": -1,
    "forwardTrafficCX": false,
    "forwardTrafficSFR": false,
    "hardwareID": "Management0/0",
    "interfaceDesc": "Configured again",
    "ipAddress": {
        "ip": {
            "kind": "IPv4Address",
            "value": "172.16.112.110"
        },
        "kind": "StaticIP",
        "netMask": {
            "kind": "IPv4NetMask",
            "value": "255.255.255.0"
        }
    },
    "ipv6Info": {
        "autoConfig": false,
        "dadAttempts": 1,
        "enabled": false,
        "enforceEUI64": false,
        "ipv6Addresses": [],
        "kind": "object#Ipv6InterfaceInfo",
        "managedAddressConfig": false,
        "nDiscoveryPrefixList": [],
        "nsInterval": 1000,
        "otherStatefulConfig": false,
        "reachableTime": 0,
        "r

Change the description and then check it again, printing out the name before and the name after.

In [86]:
payload = "{\n  \"kind\": \"object#MgmtInterface\",\n  \"interfaceDesc\": \"Configured again\"\n}"

requests.request("PATCH", url, data=payload, headers=headers, verify=False)

response2 = requests.request("GET", url, headers=headers, verify=False)

print "Name Before = " + json.loads(response1.text)['interfaceDesc']
print "Name After = " + json.loads(response2.text)['interfaceDesc']


Name Before = Configured again
Name After = Configured again


## RESTCONF on CSR1kV
Here is a script for getting interface stats using the ietf-interfaces YANG model over RESTCONF.  The code was generated in Postman.

In [87]:
url = "http://172.16.112.101:8008/api/operational/interfaces-state/interface/GigabitEthernet1"

querystring = {"deep":""}

headers = {
    'authorization': "Basic Y2lzY286Y2lzY28=",
    'content-type': "application/vnd.yang.data+json",
    'accept': "application/vnd.yang.data+json",
    'cache-control': "no-cache",
    'postman-token': "e6df43c3-bb08-98a9-8faa-b5821f6617f4"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{
  "ietf-interfaces:interface": {
    "name": "GigabitEthernet1",
    "oper-status": "up",
    "phys-address": "00:0c:29:5a:cb:71",
    "speed": 1000000000,
    "statistics": {
      "in-octets": 6699195,
      "in-broadcast-pkts": 0,
      "in-multicast-pkts": 0,
      "in-discards": 0,
      "in-errors": 0,
      "out-octets": 2857671,
      "out-discards": 0,
      "out-errors": 0,
      "ietf-interfaces-ext:in-pkts": 26063,
      "ietf-interfaces-ext:out-pkts": 4481
    },
    "cisco-ethernet:ethernet": {
    }
  }
}



## YANG


Exposing their data models gives routers a way to describe themselves to clients. As we saw earlier, routers send a list of the models they support in the initial <hello> message. To recap:

In [88]:
import re

for c in xe.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print model.group(1)

CISCO-CBP-TC-MIB
RSVP-MIB
ietf-packet-fields
CISCO-ENTITY-EXT-MIB
cisco-qos-common
CISCO-NETSYNC-MIB
SNMPv2-MIB
cisco-environment
SNMP-NOTIFICATION-MIB
CISCO-CBP-TARGET-TC-MIB
DISMAN-EVENT-MIB
MPLS-LSR-STD-MIB
CISCO-FRAME-RELAY-MIB
CISCO-ATM-PVCTRAP-EXTN-MIB
tailf-webui
CISCO-PRODUCTS-MIB
CISCO-UBE-MIB
ietf-access-control-list
MPLS-LDP-GENERIC-STD-MIB
CISCO-AAA-SESSION-MIB
tailf-common-monitoring
CISCO-RF-MIB
ENTITY-MIB
IP-FORWARD-MIB
CISCO-SUBSCRIBER-SESSION-TC-MIB
cisco-efp-stats
CISCO-PPPOE-MIB
cisco-qos-action-qlimit-cfg
CISCO-BULK-FILE-MIB
iana-crypt-hash
CISCO-NETFLOW-MIB
EtherLike-MIB
CISCO-EIGRP-MIB
cisco-table-map
SNMP-MPD-MIB
CISCO-ENHANCED-MEMPOOL-MIB
CISCO-CDP-MIB
ietf-interfaces
cisco-odm
CISCO-LAG-MIB
LLDP-MIB
tailf-confd-monitoring
common-mpls-types
CISCO-IETF-PW-MIB
CISCO-PROCESS-MIB
DS3-MIB
CISCO-OSPF-MIB
cisco-platform-software
cisco-ip-sla-stats-dev
CISCO-IETF-MPLS-TE-EXT-STD-03-MIB
cisco-flow-monitor
CISCO-OSPF-TRAP-MIB
ietf-diffserv-target
CISCO-NTP-MIB
CISCO-IETF-

### Schema Discovery via NETCONF
But what exactly is in those models?  NETCONF gives us the ability to retrieve the contents of a model real-time using the <get-schema> operation and the ncclient library provides a nice, simple function for that.

In [89]:
c = xe.get_schema('ietf-interfaces')
print c.data

module ietf-interfaces {

  namespace "urn:ietf:params:xml:ns:yang:ietf-interfaces";
  prefix if;

  import ietf-yang-types {
    prefix yang;
  }

  organization
    "IETF NETMOD (NETCONF Data Modeling Language) Working Group";

  contact
    "WG Web:   <http://tools.ietf.org/wg/netmod/>
     WG List:  <mailto:netmod@ietf.org>

     WG Chair: Thomas Nadeau
               <mailto:tnadeau@lucidvision.com>

     WG Chair: Juergen Schoenwaelder
               <mailto:j.schoenwaelder@jacobs-university.de>

     Editor:   Martin Bjorklund
               <mailto:mbj@tail-f.com>";

  description
    "This module contains a collection of YANG definitions for
     managing network interfaces.

     Copyright (c) 2014 IETF Trust and the persons identified as
     authors of the code.  All rights reserved.

     Redistribution and use in source and binary forms, with or
     without modification, is permitted pursuant to, and subject
     to the license terms contained in, the Simplified BSD Lice

### Schema Download from Github
Now, while it's nice to be able to do a get-schema and see a router's supported models in real-time, you may not always have access to a router or you may want to work offline with models.  Github is your friend.  
```
scadora$ git clone https://github.com/YangModels/yang.git
Cloning into 'yang'...
remote: Counting objects: 3583, done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 3583 (delta 7), reused 0 (delta 0), pack-reused 3568
Receiving objects: 100% (3583/3583), 7.41 MiB | 423 KiB/s, done.
Resolving deltas: 100% (1821/1821), done.

scadora$ cd yang
scadora$ ls
README.md	experimental	standard	tools		vendor
```

### A Usable View with Pyang
Getting a model is well and good, but generally only brave souls and model developers actually read .yang files.  They are just too verbose.  To get an idea   [Pyang](https://github.com/mbj4668/pyang) is a utility that makes models easier to digest by presenting an abbreviated "tree" view (among other things).  
```
scadora$ pyang -f tree ietf-interfaces@2014-05-08.yang
module: ietf-interfaces
   +--rw interfaces
   |  +--rw interface* [name]
   |     +--rw name                        string
   |     +--rw description?                string
   |     +--rw type                        identityref
   |     +--rw enabled?                    boolean
   |     +--rw link-up-down-trap-enable?   enumeration {if-mib}?
   +--ro interfaces-state
      +--ro interface* [name]
      <snipped for brevity>
```



#### Geek Alert
While it's written in Python, Pyang is a standalone utilty that is meant to be executed from a shell.  Just for fun, let's use the Python ```subprocess``` module that lets us execute a shell command and get the output back:

In [90]:
from subprocess import Popen, PIPE, STDOUT

c = xe.get_schema('cisco-process-cpu')

p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE) 
print(p.communicate(input=c.data)[0])

module: cisco-process-cpu
   +--ro cpu-usage
      +--ro cpu-utilization
      |  +--ro five-seconds?        uint8
      |  +--ro five-seconds-intr?   uint8
      |  +--ro one-minute?          uint8
      |  +--ro five-minutes?        uint8
      +--ro process-cpu-usage
         +--ro process* [pid name]
            +--ro pid                 uint32
            +--ro name                string
            +--ro tty?                uint16
            +--ro total-run-time?     uint64
            +--ro invocation-count?   uint32
            +--ro avg-run-time?       uint64
            +--ro five-seconds?       decimal64
            +--ro one-minute?         decimal64
            +--ro five-minutes?       decimal64



Geek alert over.  All done with our NETCONF session to XE, so tidy up.

In [91]:
xe.close_session()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:7cbe804a-6717-46a6-a739-ab8abe107ca3" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

### Generating JSON/XML with YANG-Explorer

[YANG-Explorer](https://github.com/CiscoDevNet/yang-explorer) is a GUI-based tool that lets you explore YANG models and generate XML and code.  Note the "Beta" status (getting dependencies right for install can be tricky)

After installing it, go to http://localhost:8088/static/YangExplorer.html

### YDK-PY

YDK-PY has generated python classes that correspond directly to YANG models. It is based on ncclient and pyang libraries.  

#### Connect to router
If this looks like ncclient, that's because YDK is based on ncclient!

In [92]:
from ydk.providers import NetconfServiceProvider
from ydk.services import CRUDService

rpc_service = CRUDService()

xr = NetconfServiceProvider(address='172.16.112.1',
                                     port=2223,
                                     username='vagrant',
                                     password='vagrant',
                                     protocol = 'ssh')

#### First example: get interface config
This bit of code reads the interface configuration and prints out the name and description of each interface with a list comprehension (fancy Python syntax).  

In [93]:
import ydk.models.ifmgr.Cisco_IOS_XR_ifmgr_cfg as ifmgr_cfg #Import the classes for the model

q = ifmgr_cfg.InterfaceConfigurations().InterfaceConfiguration() #Define the query

intfs = rpc_service.read(xr, q) #Read the data from the device

print [{"name":i.interface_name,"description":i.description} for i in intfs] #Print the results


[{'name': 'Loopback0', 'description': 'An Awesome Loopback!'}, {'name': 'MgmtEth0/RP0/CPU0/0', 'description': None}]


#### Second example: get interface stats
Basically the same code as above with a different model to get interface stats and a "for" loop instead of a list comprehension.

In [94]:
import ydk.models.infra.Cisco_IOS_XR_infra_statsd_oper as statsd_oper #Import model classes

q = statsd_oper.InfraStatistics().Interfaces().Interface() #Define the query

intfs = rpc_service.read(xr, q) #Read the data

for i in intfs: #Print the results
    print "\n", i.interface_name
    for key,counter in (vars(i.latest.generic_counters)).items():
        if type(counter)==int or type(counter)==long:
            print key, counter


Null0
output_drops 0
availability_flag 0
carrier_transitions 0
broadcast_packets_received 0
output_underruns 0
input_errors 0
bytes_received 0
resets 0
broadcast_packets_sent 0
giant_packets_received 0
input_drops 0
applique 0
unknown_protocol_packets_received 0
seconds_since_packet_sent 4294967295
crc_errors 0
last_data_time 1467222288
output_errors 0
last_discontinuity_time 1467199574
seconds_since_last_clear_counters 0
output_buffers_swapped_out 0
throttled_packets_received 0
input_queue_drops 0
input_ignored_packets 0
parity_packets_received 0
multicast_packets_sent 0
framing_errors_received 0
packets_sent 0
multicast_packets_received 0
input_overruns 0
packets_received 0
bytes_sent 0
runt_packets_received 0
output_queue_drops 0
input_aborts 0
seconds_since_packet_received 4294967295
output_buffer_failures 0

MgmtEth0/RP0/CPU0/0
output_drops 0
availability_flag 0
carrier_transitions 39
broadcast_packets_received 15
output_underruns 0
input_errors 0
bytes_received 745258
resets 0
b

All done with XR.  Tidy up and close the session.

In [95]:
xr.close()